# Guía práctica: Chatbot IA con RAG

Este notebook es una versión tutorial y limpia del examen original.
Presenta tres stages bien definidos:

1. Ingesta y embeddings
2. Índice y recuperación (RAG)
3. Chatbot y generación (fallback)

Cada stage incluye explicación, código reutilizable y pruebas mínimas. El notebook está en español y preparado para ejecutarse en modo "mock" (sin clave API) o en modo real si se configura `OPENAI_API_KEY`.


## 1) Instalación y entorno

Instalación rápida (PowerShell) y comprobación de versiones. Si no tienes `OPENAI_API_KEY`, el notebook funcionará en modo `mock`.

- Requisitos mínimos en `requirements.txt`.
- Variables de entorno: `OPENAI_API_KEY` (opcional para modo mock).


```powershell
# PowerShell: crear virtualenv e instalar dependencias
python -m venv .venv; .\.venv\Scripts\Activate.ps1
pip install -r requirements.txt
```

```python
# Comprobar entorno y variables (celda ejecutable)
import sys, os
import importlib

print('Python', sys.version.split()[0])
print('OPENAI_API_KEY present:', 'OPENAI_API_KEY' in os.environ)

# Mostrar versiones de paquetes clave si están instalados
for pkg in ('numpy','pandas','openai'):
    try:
        mod = importlib.import_module(pkg)
        print(pkg, 'version', getattr(mod, '__version__', 'unknown'))
    except Exception:
        print(pkg, 'no instalado')
```


## 2) Estructura y limpieza de redundancias

En esta sección consolidamos la configuración y eliminamos duplicados del examen original:
- Unificar la inicialización del cliente OpenAI en un solo bloque.
- Definir constantes (modelos, paths) en una sola celda.
- Reemplazar llamadas ad-hoc por funciones reutilizables (helpers).


In [ ]:
# Configuración unificada y utilidades
import os
import time
import json
import math
import logging
from typing import List, Dict, Any

import numpy as np
import pandas as pd

# Configuración básica
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# Paths (ajustar según el repo)
DATA_DIR = '.'
KNOWLEDGE_CSV = os.path.join(DATA_DIR, 'knowledge_base.csv')
PROCESSED_QUERIES = os.path.join(DATA_DIR, 'processed_queries.csv')
PREDEFINED_RESPONSES = os.path.join(DATA_DIR, 'predefined_responses.json')
CHATBOT_RESPONSES = os.path.join(DATA_DIR, 'chatbot_responses.json')

# Modelos y parámetros
EMBEDDING_MODEL = 'text-embedding-3-small'
GPT_MODEL = 'gpt-3.5-turbo'
SIMILARITY_THRESHOLD = 0.1
BATCH_SIZE = 64

# Modo: si no hay clave de OpenAI se usa MOCK
USE_MOCK = 'OPENAI_API_KEY' not in os.environ

# Helpers: retry wrapper
def retry(func, retries=3, delay=2, *args, **kwargs):
    for attempt in range(retries):
        try:
            return func(*args, **kwargs)
        except Exception as e:
            logger.warning(f"Attempt {attempt+1}/{retries} failed: {e}")
            if attempt < retries - 1:
                time.sleep(delay)
            else:
                raise

# Cosine similarity helper
def cosine_similarity_matrix(a: np.ndarray, b: np.ndarray) -> np.ndarray:
    """Return cosine similarities between each row in a and vector b (or rows in b)."""
    if a.ndim == 1:
        a = a.reshape(1, -1)
    if b.ndim == 1:
        b = b.reshape(1, -1)
    a_norm = np.linalg.norm(a, axis=1, keepdims=True)
    b_norm = np.linalg.norm(b, axis=1, keepdims=True)
    return (a @ b.T) / (a_norm * b_norm.T + 1e-10)

# Mock embedding generator (deterministic) for testing sin API
import hashlib

def _mock_embedding(text: str, dim: int = 1536) -> np.ndarray:
    # Deterministic pseudo-embedding using hash
    h = hashlib.sha256(text.encode('utf-8')).digest()
    vec = np.frombuffer(h, dtype=np.uint8).astype(np.float32)
    vec = np.pad(vec, (0, max(0, dim - vec.size)), mode='wrap')[:dim]
    return vec / (np.linalg.norm(vec) + 1e-10)

# Wrapper for embeddings (mock or real)
def make_embeddings(texts: List[str], model: str = EMBEDDING_MODEL, batch_size: int = BATCH_SIZE) -> List[np.ndarray]:
    """Genera embeddings: en modo mock usa _mock_embedding, si hay clave llama a la API real.
    Devuelve una lista de numpy arrays."""
    if USE_MOCK:
        return [_mock_embedding(t) for t in texts]
    else:
        # Aquí se colocaría la llamada a la API real (ej: client.embeddings.create)
        from openai import OpenAI
        client = OpenAI()
        embeddings = []
        for i in range(0, len(texts), batch_size):
            batch = texts[i:i+batch_size]
            resp = client.embeddings.create(input=batch, model=model)
            embeddings.extend([np.array(item.embedding) for item in resp.data])
        return embeddings


## Stage 1 — Ingesta de datos y creación de embeddings

En este stage mostramos cómo cargar `knowledge_base.csv`, preprocesar y crear embeddings de manera eficiente con batching y caching.


In [ ]:
# Funciones para Stage 1: carga y preprocesado

def load_documents_csv(path: str) -> pd.DataFrame:
    df = pd.read_csv(path)
    expected_cols = {'document_id', 'document_text'}
    if not expected_cols.issubset(set(df.columns)):
        raise ValueError(f"CSV debe contener las columnas: {expected_cols}")
    return df


def chunk_text(text: str, chunk_size: int = 500, overlap: int = 50) -> List[str]:
    tokens = text.split()
    if len(tokens) <= chunk_size:
        return [text]
    chunks = []
    start = 0
    while start < len(tokens):
        end = min(start + chunk_size, len(tokens))
        chunk = ' '.join(tokens[start:end])
        chunks.append(chunk)
        if end == len(tokens):
            break
        start = end - overlap
    return chunks

# Ejemplo: crear embeddings y guardarlos (modo mock si no hay API key)

def stage1_create_embeddings(input_csv: str = KNOWLEDGE_CSV, output_json: str = 'knowledge_embeddings.json') -> None:
    df = load_documents_csv(input_csv)
    texts = df['document_text'].tolist()
    embeddings = make_embeddings(texts)
    df['embedding_vector'] = [vec.tolist() for vec in embeddings]
    # Guardar en JSON simplificado
    records = df.to_dict(orient='records')
    with open(output_json, 'w', encoding='utf-8') as f:
        json.dump(records, f, indent=2, ensure_ascii=False)
    logger.info(f'Stage1: guardado {len(records)} embeddings en {output_json}')

# Test rápido (si existe el archivo)
try:
    if os.path.exists(KNOWLEDGE_CSV):
        print('Se detectó knowledge_base.csv; puedes ejecutar stage1_create_embeddings()')
    else:
        print('No se detectó knowledge_base.csv; el notebook puede usarse en modo mock.')
except Exception as e:
    print('Error comprobando archivos:', e)


## Stage 2 — Construcción del índice y recuperación (RAG)

Esta sección muestra un vector store simple usando numpy y persistencia a JSON (ligero, para demo). Para producción, reemplazar por FAISS/Annoy/Pinecone.


In [ ]:
# Stage 2: build vector store simple (numpy)

def build_vector_store_from_embeddings(records: List[Dict[str, Any]], persist_path: str = 'vector_store.json') -> Dict[str, Any]:
    """Espera records con keys: document_id, document_text, embedding_vector"""
    vectors = np.array([np.array(r['embedding_vector']) for r in records])
    texts = [r.get('document_text') for r in records]
    meta = [{'document_id': r['document_id']} for r in records]
    store = {
        'vectors': vectors.tolist(),
        'texts': texts,
        'meta': meta
    }
    with open(persist_path, 'w', encoding='utf-8') as f:
        json.dump(store, f, indent=2, ensure_ascii=False)
    logger.info(f'Persistido vector store en {persist_path} con {len(texts)} entradas')
    return store


def load_vector_store(persist_path: str = 'vector_store.json') -> Dict[str, Any]:
    with open(persist_path, 'r', encoding='utf-8') as f:
        store = json.load(f)
    store['vectors'] = np.array(store['vectors'])
    return store


def retrieve(store: Dict[str, Any], query: str, top_k: int = 3) -> List[Dict[str, Any]]:
    q_emb = make_embeddings([query])[0]
    sims = cosine_similarity_matrix(store['vectors'], q_emb).ravel()
    idx = np.argsort(sims)[::-1][:top_k]
    results = []
    for i in idx:
        results.append({'text': store['texts'][i], 'meta': store['meta'][i], 'score': float(sims[i])})
    return results


## Stage 3 — Chatbot (RAG + fallback a generación)

En este stage ensamblamos el pipeline: embedding de la query, recuperación (RAG), construir prompt con contexto y generar respuesta con LLM si hace falta.


In [ ]:
# Pipeline end-to-end y funciones de generación

def format_context(docs: List[Dict[str, Any]]) -> str:
    return "\n\n".join([f"- {d['text']} (score={d['score']:.3f})" for d in docs])


def build_prompt(context: str, user_query: str, system_template: str = None) -> str:
    if system_template is None:
        system_template = (
            "Eres un asistente de soporte al cliente. Usa el contexto para responder con precisión y brevedad.\n\nContexto:\n{context}\n\nPregunta:\n{query}\n\nRespuesta:" 
        )
    return system_template.format(context=context, query=user_query)


def mock_generate(prompt: str) -> str:
    # Respuesta mock para demo
    return "[MOCK GENERATED RESPONSE] Basado en el contexto: " + prompt[:200]


def pipeline_end_to_end(store: Dict[str, Any], query: str, top_k: int = 3):
    docs = retrieve(store, query, top_k=top_k)
    context = format_context(docs)
    if docs and docs[0]['score'] >= SIMILARITY_THRESHOLD:
        # RAG: usar la respuesta recuperada
        return {'source': 'RAG', 'answer': docs[0]['text'], 'score': docs[0]['score']}
    else:
        # Fallback a generación (mock o real)
        prompt = build_prompt(context, query)
        if USE_MOCK:
            answer = mock_generate(prompt)
        else:
            from openai import OpenAI
            client = OpenAI()
            resp = client.chat.completions.create(model=GPT_MODEL, messages=[{'role':'system','content':'You are a helpful assistant.'},{'role':'user','content':prompt}])
            answer = resp.choices[0].message.content
        return {'source': 'GEN', 'answer': answer, 'score': 0.0}

# Demo mini end-to-end con store simulado si no hay persistido
if os.path.exists('vector_store.json'):
    store = load_vector_store('vector_store.json')
else:
    # crear un store de ejemplo con textos sencillos
    sample_records = [
        {'document_id': 1, 'document_text': 'Nuestro horario de soporte es 9-17 L-V.', 'embedding_vector': make_embeddings(['Nuestro horario de soporte es 9-17 L-V.'])[0].tolist()},
        {'document_id': 2, 'document_text': 'Para devoluciones, contacte a soporte en 30 días.', 'embedding_vector': make_embeddings(['Para devoluciones, contacte a soporte en 30 días.'])[0].tolist()}
    ]
    build_vector_store_from_embeddings(sample_records, persist_path='vector_store.json')
    store = load_vector_store('vector_store.json')

# Ejemplos
print(pipeline_end_to_end(store, '¿Cuáles son sus horas de soporte?'))
print(pipeline_end_to_end(store, '¿Cómo solicito una devolución por producto defectuoso?'))


## Edge cases y consideraciones

- Rate limits: usar batching y reintentos.
- Normalización: comprobar normas de longitud y normalizar vectores.
- Archivos faltantes: validar paths antes de ejecutar.
- Diferencias de dimensión en embeddings: asegurar dim consistente.
- Caching y persistencia del índice para ahorrar costes.


In [ ]:
## Pruebas rápidas (unitarias) — demo con asserts

# Test: cosine_similarity_matrix
import numpy as np

a = np.array([[1.0, 0.0, 0.0], [0.0, 1.0, 0.0]])
b = np.array([1.0, 0.0, 0.0])
res = cosine_similarity_matrix(a, b).ravel()
assert abs(res[0] - 1.0) < 1e-6
assert abs(res[1] - 0.0) < 1e-6
print('Test de similitud coseno OK')

# Test: chunk_text
s = ' '.join([f'pal{i}' for i in range(1200)])
chunks = chunk_text(s, chunk_size=200, overlap=20)
assert isinstance(chunks, list) and len(chunks) > 1
print('Test chunk_text OK')
